# LLM with Web Search and Crawl

풍부한 컨텍스틀 활용하기 위해 구글검색 된 결과의 최상위 n개의 페이지를 크롤링하여 LLM에 제공하기 위한 코드입니다.



In [61]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True)

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-08-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

bs4 or scrapy?

In [ ]:
import requests
import json
import scrapy
from bs4 import BeautifulSoup
import httpx
import asyncio

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

def extract_text_and_tables_by_bs4(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Extract main text
    paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]
    text = "\n".join(paragraphs)
    return text


async def extract_text_and_tables_async(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    async with httpx.AsyncClient(timeout=2) as client:
        try:
            response = await client.get(url, headers=headers)
            response.raise_for_status()
        except httpx.HTTPError as e:
            print(f"Request failed: {e}")
            return ""

        selector = scrapy.Selector(text=response.text)
        paragraphs = [p.strip() for p in selector.css('p::text').getall() if p.strip()]
        text = "\n".join(paragraphs)
        return text

async def add_context_async(top_urls = []):
    async def gather_contexts():
        tasks = [extract_text_and_tables_async(url) for url in top_urls]
        results = await asyncio.gather(*tasks)
        return results
    return await gather_contexts()

def google_search(query, num=5):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "num": num, 
        "locale": "ko",  # 한국어로 검색
        "siteSearch": "samsung.com",
        "siteSearchFilter": "e",
    }
    response = requests.get(url, params=params)
    results = response.json()
    return results.get("items", [])

       
QUERY_REWRITE_PROMPT = """
        너는 구글 검색과 LLM을 위한 질문 재작성 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해줘.

        Google Search용 Query Rewrite 프롬프트
                사용자의 질문을 실제 구글 검색창에 입력할 수 있도록, 명확하고 간결한 검색어로 재작성해줘. 불필요한 문장이나 맥락 설명은 빼고, 핵심 키워드와 관련 정보를 중심으로 검색에 최적화된 형태로 만들어줘.
        LLM Query용 Rewrite 프롬프트
                사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해줘. 필요한 경우, 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어줘.

        예시:   
        * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
        * 구글 검색용 재작성: 삼성전자 3구 이상 인덕션 추천
        * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

        출력포맷. 반드시 json 형식으로 출력해줘.:
        {"google_search": "구글 검색용 재작성", "llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())







In [78]:
from IPython.display import Markdown, display
from datetime import datetime

RESULTS_COUNT = 5

inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    "삼성전자 25년 제품이 작년 대비 좋아진것은",
    "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]

# inputs = ["삼성전자 25년 제품이 작년 대비 좋아진것은"]

for input in inputs:
    print(f"Original Input: {input}")
    
    query_rewrite = rewrite_query_for_search_and_llm(input, client)
    print(f"Google Search Query: {query_rewrite['google_search']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    results = google_search(query_rewrite['google_search'], RESULTS_COUNT)
    print(f"Google Search Results: {len(results)}")
    top_urls = [results[i]["link"] for i in range(len(results))]

    contexts = await add_context_async(top_urls)

    # for i, context in enumerate(contexts):
    #     print(f"Context {i+1}: {context[:1000]}...")  # Print first 1000 chars of each context
    #     print("\n--- End of Context ---\n")

    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day

    system_prompt = "너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 답변은 마크다운으로 작성해줘."
    user_prompt = f"""
        너는 아래 제공하는 구글에서 검색한 컨텍스트를 바탕으로 질문에 대한 답변을 제공해야 해. 컨텍스트를 최대한 활용하여 풍부하게 답변을 해야해. 
        현재는 {year}년 {month}월 {day}일이므로 최신의 데이터를 기반으로 답변을 해줘.

        구글에서 제공한 컨텍스트: {contexts}
        질문: {query_rewrite['llm_query']}
        """

    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=[{"role": "system", "content": system_prompt},
                 {"role": "user", "content": user_prompt}],
        top_p=0.9,
        max_tokens=1500
    )

    display(Markdown(response.choices[0].message.content))

Original Input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Google Search Query: 삼성전자 3구 이상 인덕션 추천
LLM Query: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.
Google Search Query: 삼성전자 3구 이상 인덕션 추천
LLM Query: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.
Google Search Results: 5
Google Search Results: 5


삼성전자 인덕션 중 2구 모델이 아닌 3구 이상 또는 다양한 화구 수를 가진 제품 중에서 추천드릴 만한 최신 인기 모델과 그 주요 기능, 장점은 아래와 같습니다.

---

## 1. 삼성전자 비스포크 AI 3구 인덕션 NZ63DB607CF

- **출력:** 대화구 3,400W / 중화구 2,600W / 소화구 1,200W (상위 10% 최상급 화력)
- **주요 기능:**
  - AI 넘침 방지 모드 탑재
  - 저소음 기능(DCN)
  - 대화형 알림창 지원으로 조작 편리
  - 심플하고 커진 조작부
  - 프레임리스 디자인, 얇은 두께로 식기세척기와 설치 시 공간 활용 우수
- **장점:**
  - 최상급 화력과 뛰어난 편의 기능 탑재
  - 가격은 100만 원 미만으로 이전 대비 가성비 대폭 향상
  - 삼성 비스포크 라인의 세련된 디자인과 사용자 친화적 UI

---

## 2. 삼성전자 비스포크 AI 3구 인덕션 NZ63DB607CABS

- **출력:** NZ63DB607CF와 동일 (대화구 3,400W 등)
- **주요 기능:**
  - 스마트컨트롤 지원 (외부에서도 원격 전원 제어 가능)
  - 상판은 업계 1위 브랜드 SCHOTT 제품 사용 (고급 상판)
- **장점:**
  - 비스포크 AI 3구 모델 중 한 단계 상위급 성능과 품질
  - 스마트폰 연동 원격 제어 가능
  - 상판 내구성과 디자인 우수
- **가격:** 96 ~ 115만원 (2025년 5월 기준)

---

## 3. 삼성전자 NZ63D650BXE (비스포크 전작 모델)

- **출력:** 대화구 3,400W, 중화구 2,600W, 소화구 1,200W (동급 화력)
- **주요 기능:**
  - AI 끓음 감지 기능 미탑재
  - 각 화구별 개별 조작부 3개 탑재
  - 두께가 비교적 두꺼움
- **장점:**
  - 동급 출력에 비해 가격 저렴 (66 ~ 79만원)
  - 예산 절약하면서 성능 유지 가능
  - 기본적인 비스포크 디자인과 기능 제공

---

## 4. 삼성전자 NZ63B4026AK (삼성 3구 인덕션 중 최고 가성비 모델)

- **출력:** 픽 제품과 동일, 대화구 3,400W
- **주요 기능:**
  - 상판 색상 블랙
  - 대화형 알림창 미탑재
- **장점:**
  - 매우 저렴한 가격대(55 ~ 65만원)
  - 가성비 뛰어난 출력과 기본 기능 제공
  - 디자인에 크게 신경쓰지 않는 사용자에게 적합

---

## 5. 삼성전자 하이브리드 인덕션 NZ63T4507AK (인덕션 2구 + 하이라이트 1구 구성)

- **출력:** 인덕션 대화구 3,300W / 중화구 1,300W / 하이라이트 1,050W
- **주요 기능:**
  - 인덕션과 하이라이트의 장점 결합, 용기 제한 없이 사용 가능
  - 원터치 조작, 타이머, 앱 연동, 용기효율알림 등 편의 기능 충실
  - 상판은 독일 SCHOTT 제품 탑재
- **장점:**
  - 인덕션과 하이라이트 혼용으로 용기 제약 완화
  - 높은 출력과 가성비 좋은 가격대
  - 삼성 프리미엄 상판으로 내구성 및 디자인 우수
- **가격:** 110 ~ 132만원

---

# 요약

| 모델명                    | 화구 수/구성            | 최대 출력 (대화구 기준) | 주요 장점                                      | 가격대 (2025년 5월)      |
|-------------------------|---------------------|----------------------|--------------------------------------------|---------------------|
| NZ63DB607CF (비스포크 AI)    | 3구 (인덕션)            | 3,400W               | AI 넘침 방지, 저소음, 프레임리스, 가성비 우수            | 100만원 미만          |
| NZ63DB607CABS (비스포크 AI 상위) | 3구 (인덕션)            | 3,400W               | 스마트컨트롤, SCHOTT 상판, 고급 기능                   | 96 ~ 115만원         |
| NZ63D650BXE (비스포크 전작)    | 3구 (인덕션)            | 3,400W               | 저렴한 가격, 기본 기능 충실                           | 66 ~ 79만원          |
| NZ63B4026AK (가성비 모델)     | 3구 (인덕션)            | 3,400W               | 매우 저렴, 기본 출력 유지                            | 55 ~ 65만원          |
| NZ63T4507AK (하이브리드)      | 3구 (인덕션 2구 + 하이라이트 1구) | 3,300W               | 용기 제약 없음, SCHOTT 상판, 앱 연동                   | 110 ~ 132만원        |

---

### 결론

- **최상급 성능과 최신 기능을 원하면** NZ63DB607CF 모델 추천드립니다. 가성비도 뛰어납니다.
- **스마트컨트롤과 최고급 상판을 원하면** NZ63DB607CABS가 좋습니다.
- **가격 부담을 줄이고 싶다면** NZ63D650BXE 또는 NZ63B4026AK 모델이 가성비가 좋습니다.
- **인덕션과 하이라이트를 함께 쓰고 싶으면** NZ63T4507AK 하이브리드 모델이 탁월합니다.

필요하신 기능과 예산에 맞게 선택하시면 후회 없는 선택이 될 것입니다!

추가 궁금한 점이나 다른 모델 문의 주시면 상세히 안내해 드리겠습니다. 감사합니다!

Original Input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Google Search Query: 삼성전자 TV 추천 부모님 선물
LLM Query: 부모님께 선물하기 좋은 삼성전자 TV 모델을 추천해 주세요. 내구성, 화질, 사용 편의성 등을 고려한 제품 정보도 함께 알려주세요.
Google Search Query: 삼성전자 TV 추천 부모님 선물
LLM Query: 부모님께 선물하기 좋은 삼성전자 TV 모델을 추천해 주세요. 내구성, 화질, 사용 편의성 등을 고려한 제품 정보도 함께 알려주세요.
Google Search Results: 5
Google Search Results: 5
Request failed: 
Request failed: 


안녕하세요! 부모님께 선물하기 좋은 삼성전자 TV 모델을 내구성, 화질, 사용 편의성 등을 고려해 추천드리겠습니다.

---

## 추천 모델: 삼성전자 Neo QLED 8K (예: QN900B 시리즈)

### 1. **화질**
- **Neo QLED 기술**: 미세 LED 광원으로 뛰어난 명암비와 선명한 색감을 구현합니다.
- **8K 해상도**: 7680 x 4320 픽셀로 극강의 디테일과 생생한 화면을 제공해 영화나 방송 시청 시 몰입감을 높여줍니다.
- **AI 업스케일링**: 낮은 해상도 영상도 자동으로 8K급 화질로 개선해줍니다.

### 2. **내구성**
- 삼성전자의 프리미엄 제품군으로 엄격한 품질 관리와 내구성을 자랑합니다.
- 열 방출 설계 및 안정적인 부품 사용으로 장기간 안정적인 성능을 유지할 수 있습니다.

### 3. **사용 편의성**
- **스마트 허브 탑재**: 다양한 OTT 앱과 콘텐츠에 쉽게 접근할 수 있습니다.
- **음성 인식**: 빅스비, 구글 어시스턴트, 아마존 알렉사와 연동되어 음성으로 TV를 제어할 수 있어 사용이 편리합니다.
- **심플한 리모컨 디자인**: 최소한의 버튼으로 조작이 쉬워 부모님께 적합합니다.
- **스마트싱스(SmartThings) 호환**: 집안의 스마트 가전과 연동해 편리한 생활 환경을 제공합니다.

---

## 추가 추천 모델: 삼성 QLED 4K (예: QN90B 시리즈)

- **화질**: 4K 해상도, 뛰어난 밝기와 명암비, 현실감 있는 색감을 제공합니다.
- **가격대**: 8K 모델 대비 부담이 적어 가성비가 좋습니다.
- **편의성**: 스마트 기능 및 음성 인식 지원은 Neo QLED 8K와 유사합니다.

---

## 요약

|모델|화질|내구성|사용 편의성|추천 이유|
|-|-|-|-|-|
|Neo QLED 8K (QN900B)|최고 수준 8K, AI 업스케일링|프리미엄 내구성|음성 인식, 스마트 허브, 심플 리모컨|최신 최고급 화질과 기능을 원할 때|
|QLED 4K (QN90B)|선명한 4K, 밝기 우수|우수한 내구성|음성 인식, 스마트 허브|가격 대비 뛰어난 성능과 편리성|

---

부모님께서는 TV를 장시간 사용하실 수 있으므로, 내구성과 편의성이 뛰어나고 화질이 선명한 삼성 Neo QLED 8K 또는 QLED 4K 모델을 선물하시면 만족도가 높으실 것입니다.

더 궁금한 점 있으면 언제든 문의해주세요!

Original Input: 삼성전자 25년 제품이 작년 대비 좋아진것은
Google Search Query: 삼성전자 2025년 제품 개선점 작년 대비
LLM Query: 삼성전자의 2025년 신제품이 작년 제품에 비해 어떤 점에서 개선되었는지 주요 변화와 향상된 기능을 자세히 알려주세요.
Google Search Query: 삼성전자 2025년 제품 개선점 작년 대비
LLM Query: 삼성전자의 2025년 신제품이 작년 제품에 비해 어떤 점에서 개선되었는지 주요 변화와 향상된 기능을 자세히 알려주세요.
Google Search Results: 5
Google Search Results: 5
Request failed: Redirect response '308 Permanent Redirect' for url 'https://news.zum.com/articles/96015509'
Redirect location: '/articles/96015509/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-%EC%98%AC%ED%95%B4-hbm-%EA%B3%B5%EA%B8%89-%EC%9E%91%EB%85%84-2%EB%B0%B0-%EA%B0%9C%EC%84%A0-hbm3e-3%EC%9B%94%EB%A7%90-%EC%96%91%EC%82%B0?'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/308
Request failed: Redirect response '308 Permanent Redirect' for url 'https://news.zum.com/articles/96015509'
Redirect location: '/articles/96015509/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-%EC%98%AC%ED%95%B4-hbm-%EA%B3%B5%EA%B8%89-%EC%9E%91%EB%85%84-2%EB%B0

...

삼성전자에서 판매하는 JBL과 하만카돈(Harman Kardon) 브랜드의 스피커 및 오디오 제품은 모두 하만(Harman)이 소유한 브랜드로, 각각의 특성과 주요 차이점이 있습니다. 2025년 5월 현재 최신 정보를 기반으로 두 브랜드의 주요 특징과 차이점을 비교해 드리겠습니다.

---

## JBL과 하만카돈 스피커/오디오 제품 주요 차이점 및 특징 비교

| 구분           | JBL                                      | 하만카돈 (Harman Kardon)                    |
|----------------|------------------------------------------|----------------------------------------------|
| **브랜드 포지셔닝** | 대중적이고 활동적인 라이프스타일에 적합한 브랜드, 스포츠, 야외 활동에 강점 | 고급스러운 디자인과 뛰어난 음질을 강조하는 프리미엄 브랜드 |
| **사운드 특징**    | 강력한 베이스와 명료한 중고음으로 에너지 넘치는 사운드 제공 | 균형 잡힌 사운드와 세련된 음향으로 부드럽고 깊이 있는 음질 구현 |
| **디자인 스타일**  | 견고하고 실용적인 디자인, 휴대성과 내구성 중시               | 세련되고 미니멀리즘적인 고급 디자인, 인테리어와 조화로운 스타일 |
| **주요 제품군**    | 포터블 블루투스 스피커 (JBL Flip, Charge, Pulse 시리즈 등), 차량용 오디오 시스템 | 홈 오디오 시스템, 하이파이 스피커, AV 리시버, 고급 헤드폰 등 |
| **주요 사용처**    | 야외활동, 캠핑, 파티, 운동 등 활동적인 환경에 최적            | 홈시어터, 음악 감상, 고급 오디오 환경에 적합 |
| **기술 및 기능**   | 방수(IPX7 등급 이상), 블루투스 멀티포인트 연결, 파티부스트 등 기능 제공 | 고해상도 오디오 지원, DSP 음향 조절 기능, 세밀한 사운드 튜닝 |
| **가격대**         | 중저가에서 중가 중심, 가성비 좋은 제품이 많음                  | 중고가에서 고가 중심, 프리미엄급 가격대 |

---

## 요약

- **JBL**은 활동적인 라이프스타일을 위한 튼튼하고 강력한 베이스의 휴대용 스피커를 주력으로 하며, 가격대도 비교적 합리적입니다. 특히 야외에서 사용하기 좋은 방수 기능과 휴대성, 배터리 성능이 뛰어납니다.
- **하만카돈**은 고급스러운 인테리어와 조화를 이루는 세련된 디자인과 함께 균형 잡힌 고품질 사운드를 제공하는 프리미엄 브랜드입니다. 주로 집에서 음악 감상이나 홈시어터용으로 많이 선택됩니다.

---

필요하시면 삼성전자 공식 홈페이지나 하만 브랜드의 최신 제품 라인업을 확인하여, 원하는 사용 목적과 환경에 맞는 스피커를 선택하는 것을 추천드립니다.

Original Input: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요
Google Search Query: 갤럭시 버즈 이어버드 한쪽 페어링 방법
LLM Query: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데, 새 이어버드를 기존 기기와 어떻게 페어링하는지 단계별로 알려주세요.
Google Search Query: 갤럭시 버즈 이어버드 한쪽 페어링 방법
LLM Query: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데, 새 이어버드를 기존 기기와 어떻게 페어링하는지 단계별로 알려주세요.
Google Search Results: 5
Google Search Results: 5


갤럭시 버즈 이어버드 한쪽을 새로 구매하여 기존 기기와 페어링(커플링)하는 방법을 단계별로 안내드립니다. 아래는 갤럭시 버즈3 및 버즈3 프로 모델을 기준으로 한 셀프 커플링 방법입니다.

---

## 갤럭시 버즈 이어버드 새로 구입 후 기존 기기와 페어링 단계

### 1. 이어버드 정상 안착 확인
- 새로 구입한 이어버드를 충전 케이스 좌우 슬롯 홈에 맞게 넣습니다.
- 충전 단자에 정상적으로 안착되었는지 확인합니다.
- 충전 케이스 LED가 녹색 또는 빨간색으로 깜빡여야 합니다.

### 2. 충전 케이스 연결 버튼 누르기
- 충전 케이스 뚜껑을 연 상태로 유지합니다.
- 케이스 아래 충전 단자 옆에 위치한 연결 버튼을 7초 이상 길게 누릅니다.

### 3. 커플링 완료 확인
- 충전 케이스 LED가 파란색으로 깜빡인 후 꺼지면 커플링이 완료된 것입니다.

### 4. 모바일 기기와 연결
- 이어버드를 충전 케이스에서 꺼내고, 스마트폰 등의 모바일 기기에서 Bluetooth 설정을 통해 갤럭시 버즈를 검색하여 연결합니다.
- 만약 기존 이어버드와 새로 구입한 이어버드의 소프트웨어 버전이 다를 경우에는 페어링 모드 진입이 제한될 수 있으니, 모바일 기기의 Galaxy Wearable(갤럭시 웨어러블) 앱에서 소프트웨어 업데이트를 진행해주세요.

---

## 주의사항
- 이어버드 커플링이 제대로 되지 않으면 배터리 상태 표시등이 빨간색으로 계속 깜빡입니다. 이 경우 이어버드를 충전 케이스에서 뺐다가 다시 넣고, 7초 이상 동시에 길게 터치하여 다시 시도하세요.
- 양쪽 이어버드의 배터리 타입(종류)이 다르면 커플링 작업이 불가합니다.
- 만약 셀프 커플링이 어려운 경우 또는 서비스센터 방문이 필요한 경우, 엔지니어가 커플링 작업을 도와드립니다.
- 개인 간 거래로 구입 시에는 반드시 배터리 타입이 기존 이어버드와 같은지 확인해야 합니다.

---

필요한 경우 서비스센터 방문을 통해 이어버드 구입 및 커플링 작업을 진행할 수 있으니 참고하시기 바랍니다.  
더 궁금하신 사항이 있으면 언제든 문의해 주세요!

Original Input: 삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나
Google Search Query: 삼성전자 S25 무게 S24 비교
LLM Query: 삼성전자 S25 모델의 무게가 이전 모델인 S24와 비교했을 때 얼마나 차이가 나는지 알려주세요. 두 모델의 무게 비교와 차이점을 상세히 설명해 주세요.
Google Search Query: 삼성전자 S25 무게 S24 비교
LLM Query: 삼성전자 S25 모델의 무게가 이전 모델인 S24와 비교했을 때 얼마나 차이가 나는지 알려주세요. 두 모델의 무게 비교와 차이점을 상세히 설명해 주세요.
Google Search Results: 5
Google Search Results: 5


삼성전자 갤럭시 S25 시리즈와 이전 모델 S24 시리즈의 무게 차이에 대해 상세히 안내해 드리겠습니다.

---

## 갤럭시 S25 vs S24 무게 비교

| 모델       | S24 무게 | S25 무게 | 무게 차이 및 설명                         |
|------------|----------|----------|-----------------------------------------|
| 기본형     | 167g     | 162g     | 5g 가벼워짐. 더 얇아진 두께(7.6mm → 7.2mm)와 함께 휴대성 향상 |
| 플러스     | 190g     | 190g     | 무게 동일. 두께는 약간 얇아짐(7.7mm → 7.3mm)          |
| 울트라     | 232g     | 218g     | 14g 가벼워짐. 디자인이 더 둥글고 슬림해져 그립감 개선    |

---

## 상세 설명

### 1. 기본형 모델
- **S24 기본형 무게**는 약 167g이고, **S25 기본형은 162g**으로 약 5g 가벼워졌습니다.
- 두께도 7.6mm에서 7.2mm로 더 얇아져서 전체적으로 더욱 슬림하고 가벼운 느낌을 줍니다.

### 2. 플러스 모델
- 플러스 모델의 무게는 S24와 S25 모두 약 190g으로 동일합니다.
- 다만, 두께는 7.7mm에서 7.3mm로 약간 얇아져 디자인이 개선되었습니다.

### 3. 울트라 모델
- 울트라 모델은 **S24에서 232g**, **S25에서 218g**으로 약 14g 가벼워졌습니다.
- 디자인이 기존 각진 모서리에서 3년 만에 다시 둥근 모서리로 변경되었고, 두께도 8.6mm에서 8.2mm로 줄어들어 그립감이 많이 향상되었습니다.
- 무게 감소와 더불어 배터리 용량은 유지하면서도 그립감과 휴대성이 크게 개선된 점이 특징입니다.

---

## 종합

갤럭시 S25 시리즈는 전체적으로 무게를 줄이고 두께도 얇게 설계하여 휴대성과 사용자 경험을 향상시켰습니다. 특히 울트라 모델에서 무게가 14g이나 감소한 점이 눈에 띄며, 기본형도 5g 가벼워져 부담을 줄였습니다. 플러스 모델은 무게는 유지하였으나 두께가 얇아진 점이 개선 포인트입니다.

더욱 자세한 스펙이나 비교는 삼성 공식 사이트에서 확인하실 수 있습니다.

---

필요한 추가 정보가 있으시면 언제든 문의해 주세요!